In [99]:
import re
sample_in = 'target area: x=20..30, y=-10..-5'
my_in = 'target area: x=155..182, y=-117..-67'

# 1: (6,9) 2 (6+5, 9+8) 3 (11+4, 9+..+7) 4 (15+3, 9+..+6) 5 (18+2, 9+..+5) 6 (20+1, 9+..+4)
# 7: (21, ..+3) 8 (+2) 9 (+1) 10 (+0) 11 (+1) 12 (+2) 13 (+3) 14 (+4) 15 (+5) 16 (+6)
# 17: (+7) 18: (9+8) 19: (9+8-8) 20 :(9-9) 21: (21, 0-10)

# 1: (6, 0) 2: (6+5, 0-1) 3:(11+4, -1-2) 4:(15+3, -3-3) 5:(18+2, -6-4) = (20, -10)

# 1: (9, -2) 2: (9+8, -2-3) 3: (17+7, -5-4)

def get_input(s):
    rm = re.search('target area: x=(.*)\.\.(.*), y=(.*)\.\.(.*)', s)
    return tuple(int(i) for i in rm.groups())

print(get_input(sample_in))
print(get_input(my_in))

(20, 30, -10, -5)
(155, 182, -117, -67)


In [100]:
def prob1(s):
    xmin, xmax, ymin, ymax = get_input(s)
    assert ymin < 0 and ymax < 0, f'{ymin} and {ymax} must have been negative'
    y_ops = get_initial_vertical_options(ymin, ymax)
    for vely in y_ops:
        velx = canbedone(xmin, xmax, ymin, ymax, vely)
        if velx:
            break
    result = simulate_highest_y(vely)
    return result
def simulate_highest_y(vel_y):
    # sum 1,2..vel_y
    return int((vel_y + 1) * vel_y / 2)
def get_initial_vertical_options(ymin, ymax):
    # We can't get a vely over -(ymin-1) because the first down-step will be at -vely-1
    for vely in range(ymin + 1, 0):
        yield -vely
    print('Finished getting velocities')
def canbedone(xmin, xmax, ymin, ymax, vely):
    steps = how_many_steps(vely, ymin, ymax)
    #print(f'Trying {vely} initial {ymin}..{ymax}: {steps}')
    results = []
    for extra_step in steps:
        step = extra_step
        if vely > 0:
            step += vely * 2 + 2
        if vely == 0:
            step += 1
        velx = get_velx(xmin, xmax, step)
        if vely == -2:
            print(f'steps {step}, {velx}, {vely}, {steps}')
        if velx:
            results.extend(velx)
    return set(results)
def how_many_steps(vely, ymin, ymax):
    # (((1 + steps)* (steps) / 2) + steps * vely) between -ymax and -ymin
    result = []
    for steps in range(-ymin * 1000):
        # vely+1 + vely+2 + ... + vely+n = (vely+vely+..+vely) + (1+2+..+n)
        #                                = (n * vely       ) + ((1+n) * (n) / 2)
        intent = int((1+steps)*steps/2 + steps * vely)  # Down shot
        if vely >= 0:
            # intent = sum(range(vely, vely + steps))
            # print(steps, intent)
            if intent > -ymin:
                break
            if intent >= -ymax:
                result.append(steps)
        else:
            if intent < ymin:
                break
            if intent <= ymax:
                result.append(steps)
    return result

def get_velx(xmin, xmax, steps):
    results = []
    for velx in range(xmax + 1):
        intent = 0
        for i in range(steps):
            intent += max(velx - i, 0)
        # print(f'for {velx} applied {steps} times we get {intent}. Is it in [{xmin}, {xmax}]?')
        if intent > xmax:
            # print(f'Breaking {velx} with {intent} and {results}')
            break
        if intent >= xmin:
            results.append(velx)
    return results

test1 = prob1(sample_in)
assert test1 == 45, test1
assert prob1(my_in) == 6786, prob1(my_in)

print('ok')

ok


In [101]:
def prob2(s):
    xmin, xmax, ymin, ymax = get_input(s)
    assert ymin < 0 and ymax < 0, f'{ymin} and {ymax} must have been negative'
    y_ops = get_initial_vertical_options2(ymin, ymax)
    result = 0
    internal_points = set()
    for vely in y_ops:
        velx = canbedone(xmin, xmax, ymin, ymax, vely)
        result += len(velx)
        for x in velx:
            internal_points.add((x,vely))
    print('ok     :', IT & internal_points)
    print('extra  :', internal_points - IT)
    print('missing:', IT - internal_points)
    print('ok     :', len(IT & internal_points))
    print('extra  :', len(internal_points - IT))
    print('missing:', len(IT - internal_points))
    return result
def get_initial_vertical_options2(ymin, ymax):
    # We can't get a vely over -(ymin-1) because the first down-step will be at -vely-1
    for vely in range(ymin-10, -ymin+2):
        yield -vely
    print('Finished getting velocities')
    return
def prob2(s):
    xmin, xmax, ymin, ymax = get_input(s)
    result = 0
    for x in range(xmax + 1):
        for y in range(ymin, -ymin+1):
            if simulate(x, y, xmin, xmax, ymin, ymax):
                result += 1
    return result
def simulate(velx, vely, xmin, xmax, ymin, ymax):
    posx = 0
    posy = 0
    while posx <= xmax and posy >= ymin:
        if posx in range(xmin, xmax+1) and posy in range(ymin, ymax+1):
            return True
        posx += velx
        posy += vely
        velx = max(0, velx-1)
        vely -= 1
    return False
test2 = prob2(sample_in)
assert test2 == 112, test2
res2 = prob2(my_in)
assert res2 < 2327, res2
res2

2313

In [63]:
IT = {(tuple([int(y) for y in x.split(",")])) for x in """23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
 25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
 8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
 26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
 20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
 25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
 25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
 8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
 24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
 7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
 23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
 27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
 8,-2    27,-8   30,-5   24,-7""".replace('\n', '').split(' ') if x}
[x for x in IT][:5] + ['...']

[(7, 3), (6, 9), (26, -5), (27, -5), (21, -5), '...']